# How can we walk the LLC chain?  

* addresses are stored in various tables, as are _owners_, but in most cases the format is _weird_
* in the case of owners...their isn't a primary key in most cases.
* in the case of properties...their isn't always a primary key either.  

Here's how I'm thinking of doing it.  Create new tables:

|Table|Used For ... | Populated From...|
|---|---|---|
|masterEntity|has all of the different entities (owner, businesses, etc) that exist throughout the tables |opa_properties_publicowner_1 and owner_2|
|masterAddress|every possible address that shows up in the tables|mailing_street, address_std, location|
|MasterEntityAddress|opa_propoerties_public|\|

How is this used?
* multiple LLCs have the same mailing address (we can make the connection via the address)
* at this point, should be able to do fuzzy matching
* answer questions like :  What other properties does `A KENSINGTON JOINT LLC` own?




## Other sources we can use for address resolution
* transunion
* other jurisdiction records (FL)


|Table|Populated From | columns |
|---|---|---|
|MasterEntity|opa_properties_public |owner_1 and owner_2|
|master address|opa_properties_public|mailing_street, address_std, location|
|MasterEntityAddress|opa_propoerties_public|the 6 combinations of opa owners and addresses|


## Test Cases

This is a list of all of the problem properties that were given by the Philly team.  We can use these to build out the code and test it



In [2]:
%%sql
SELECT *
FROM davelake.philly_problempropertylist

StatementMeta(, a35b73e1-db60-485b-a961-993f84d142af, 3, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 2 fields>

In [54]:
%%sql
--master "parcels" are already available in opa_properties_public (PK:  parcel_number)
--master list of LLC text
CREATE OR REPLACE TABLE davelake.masterEntity
(
    MasterEntityId STRING ,
    NameText varchar(200)
);
--master list of address text
CREATE OR REPLACE TABLE davelake.masterAddress
(
    MasterAddressID STRING ,
    AddressText varchar(200)
);
--junction table for LLCs, parcels, and addresses
--the PK is LLC, address, and parcel...kindof.  
--in some cases the tables link the LLC (entity) to the address but the parcel_number or opa_account_num is null.  That may be a valuable relationship to maintain.  
CREATE OR REPLACE TABLE davelake.masterEntityAddress
(
    masterEntityAddressID STRING ,
    MasterEntityID STRING,
    MasterAddressID STRING,
    parcel_number varchar(200),  --associated with which parcel_number
    Notes varchar(2000) --why this row was created
);


StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 160, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Populate tables from opa_properties_public_pde

In [55]:
%%sql
--get the distinct "LLC owners" (entities) from philly_opa_properties_public_pde
INSERT INTO davelake.masterEntity 
SELECT 
    uuid() AS MasterEntityID,
    DistinctOPAOwners.NameText AS NameText
FROM 
    (
        --owner_1
        SELECT 
            DISTINCT opa.owner_1 AS NameText
        FROM davelake.philly_opa_properties_public_pde opa
        WHERE opa.owner_1 IS NOT NULL
        UNION
        --owner_2
        SELECT 
            DISTINCT opa.owner_2 AS NameText
        FROM davelake.philly_opa_properties_public_pde opa
        WHERE opa.owner_2 IS NOT NULL
    ) DistinctOPAOwners 
LEFT JOIN davelake.MasterEntity master
    ON DistinctOPAOwners.NameText = master.NameText
--Entity doesn't yet exist in master table
WHERE master.NameText IS NULL
;

--now do distinct addresses from philly_opa_properties_public_pde
--address_std, location, mailing_street are available, do all three, just in case
INSERT INTO davelake.MasterAddress 
SELECT 
    uuid() AS MasterAddressID,
    DistinctAddressesFromOPATable.AddressText 
FROM 
    (
        --location col
        SELECT 
            DISTINCT opa.location AS AddressText
        FROM davelake.philly_opa_properties_public_pde opa
        WHERE opa.location IS NOT NULL
        UNION 
        --address_std
        SELECT 
            DISTINCT opa.address_std AS AddressText
        FROM davelake.philly_opa_properties_public_pde opa
        WHERE opa.address_std IS NOT NULL
        UNION
        --mailing_street
        SELECT 
            DISTINCT opa.mailing_street AS AddressText
        FROM davelake.philly_opa_properties_public_pde opa
        WHERE opa.mailing_street IS NOT NULL
    ) DistinctAddressesFromOPATable
--where the row does not yet exist
LEFT JOIN davelake.MasterAddress master
    ON DistinctAddressesFromOPATable.AddressText = master.AddressText
WHERE master.AddressText IS NULL





StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 162, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [56]:
--now do MasterEntityAddress (junction table) for philly_opa_properties_public_pde

--1. opa.owner_1 associated with opa.location
CREATE OR REPLACE TEMPORARY VIEW Case1
AS 
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    opa.parcel_number,
    lap.MasterEntityAddressID,
    CASE WHEN lap.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'opa.owner_1:opa.location'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN lap.Notes like '%opa.owner_1:opa.location%' THEN lap.Notes
            --new entry
            ELSE CONCAT(lap.Notes ,';opa.owner_1:opa.location') END 
    END AS Notes,
    CASE WHEN lap.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM davelake.philly_opa_properties_public_pde opa
JOIN davelake.MasterEntity llc
    ON opa.owner_1 = llc.NameText
JOIN davelake.masteraddress addr
    ON opa.location = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress lap
    ON llc.MasterEntityID = lap.MasterEntityID
    AND addr.MasterAddressID = lap.MasterAddressID
    AND opa.parcel_number = lap.parcel_number
;
--updates
MERGE INTO davelake.MasterEntityAddress targ
    USING Case1
        ON targ.MasterEntityID = Case1.MasterEntityID
        AND targ.MasterAddressID = Case1.MasterAddressID
        AND targ.parcel_number = Case1.parcel_number
WHEN MATCHED AND Case1.NewEntry = 0 THEN 
    UPDATE SET targ.Notes = Case1.Notes;
--inserts
INSERT INTO davelake.MasterEntityAddress
SELECT 
    uuid() AS MasterEntityAddressID,
    Case1.MasterEntityID,
    Case1.MasterAddressID,
    Case1.parcel_number ,
    Case1.Notes
FROM  Case1
WHERE Case1.NewEntry = 1

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 165, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [57]:
--2.owner_1 associated with address_std
CREATE OR REPLACE TEMPORARY VIEW Case2
AS 
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    opa.parcel_number,
    lap.MasterEntityAddressID,
    CASE WHEN lap.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'opa.owner_1:opa.address_std'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN lap.Notes like '%opa.owner_1:opa.address_std%' THEN lap.Notes
            --new entry
            ELSE CONCAT(lap.Notes ,';opa.owner_1:opa.address_std') END 
    END AS Notes,
    CASE WHEN lap.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM davelake.philly_opa_properties_public_pde opa
JOIN davelake.MasterEntity llc
    ON opa.owner_1 = llc.NameText
JOIN davelake.masteraddress addr
    ON opa.address_std = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress lap
    ON llc.MasterEntityID = lap.MasterEntityID
    AND addr.MasterAddressID = lap.MasterAddressID
    AND opa.parcel_number = lap.parcel_number
;
--updates
MERGE INTO davelake.MasterEntityAddress targ
    USING Case2
        ON targ.MasterEntityID = Case2.MasterEntityID
        AND targ.MasterAddressID = Case2.MasterAddressID
        AND targ.parcel_number = Case2.parcel_number
WHEN MATCHED AND Case2.NewEntry = 0 THEN 
    UPDATE SET targ.Notes = Case2.Notes;
--inserts
INSERT INTO davelake.MasterEntityAddress
SELECT 
    uuid() AS MasterEntityAddressID,
    Case2.MasterEntityID,
    Case2.MasterAddressID,
    Case2.parcel_number ,
    Case2.Notes
FROM  Case2
WHERE Case2.NewEntry = 1

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 168, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [58]:
--3.owner_1 associated with mailing_street
CREATE OR REPLACE TEMPORARY VIEW Case3
AS 
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    opa.parcel_number,
    lap.MasterEntityAddressID,
    CASE WHEN lap.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'opa.owner_1:opa.mailing_street'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN lap.Notes like '%opa.owner_1:opa.mailing_street%' THEN lap.Notes
            --new entry
            ELSE CONCAT(lap.Notes ,';opa.owner_1:opa.mailing_street') END 
    END AS Notes,
    CASE WHEN lap.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM davelake.philly_opa_properties_public_pde opa
JOIN davelake.MasterEntity llc
    ON opa.owner_1 = llc.NameText
JOIN davelake.masteraddress addr
    ON opa.mailing_street = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress lap
    ON llc.MasterEntityID = lap.MasterEntityID
    AND addr.MasterAddressID = lap.MasterAddressID
    AND opa.parcel_number = lap.parcel_number
;
--updates
MERGE INTO davelake.MasterEntityAddress targ
    USING Case3
        ON targ.MasterEntityID = Case3.MasterEntityID
        AND targ.MasterAddressID = Case3.MasterAddressID
        AND targ.parcel_number = Case3.parcel_number
WHEN MATCHED AND Case3.NewEntry = 0 THEN 
    UPDATE SET targ.Notes = Case3.Notes;
--inserts
INSERT INTO davelake.MasterEntityAddress
SELECT 
    uuid() AS MasterEntityAddressID,
    Case3.MasterEntityID,
    Case3.MasterAddressID,
    Case3.parcel_number ,
    Case3.Notes
FROM  Case3
WHERE Case3.NewEntry = 1

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 171, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [59]:
--4.owner_2 associated with location
CREATE OR REPLACE TEMPORARY VIEW Case4
AS 
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    opa.parcel_number,
    lap.MasterEntityAddressID,
    CASE WHEN lap.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'opa.owner_1:opa.location'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN lap.Notes like '%opa.owner_1:opa.location%' THEN lap.Notes
            --new entry
            ELSE CONCAT(lap.Notes ,';opa.owner_1:opa.location') END 
    END AS Notes,
    CASE WHEN lap.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM davelake.philly_opa_properties_public_pde opa
JOIN davelake.MasterEntity llc
    ON opa.owner_2 = llc.NameText
JOIN davelake.masteraddress addr
    ON opa.location = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress lap
    ON llc.MasterEntityID = lap.MasterEntityID
    AND addr.MasterAddressID = lap.MasterAddressID
    AND opa.parcel_number = lap.parcel_number
;
--updates
MERGE INTO davelake.MasterEntityAddress targ
    USING Case4
        ON targ.MasterEntityID = Case4.MasterEntityID
        AND targ.MasterAddressID = Case4.MasterAddressID
        AND targ.parcel_number = Case4.parcel_number
WHEN MATCHED AND Case4.NewEntry = 0 THEN 
    UPDATE SET targ.Notes = Case4.Notes;
--inserts
INSERT INTO davelake.MasterEntityAddress
SELECT 
    uuid() AS MasterEntityAddressID,
    Case4.MasterEntityID,
    Case4.MasterAddressID,
    Case4.parcel_number ,
    Case4.Notes
FROM  Case4
WHERE Case4.NewEntry = 1

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 174, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [60]:
--5.owner_2 associated with address_std
CREATE OR REPLACE TEMPORARY VIEW Case5
AS 
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    opa.parcel_number,
    lap.MasterEntityAddressID,
    CASE WHEN lap.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'opa.owner_1:opa.address_std'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN lap.Notes like '%opa.owner_1:opa.address_std%' THEN lap.Notes
            --new entry
            ELSE CONCAT(lap.Notes ,';opa.owner_1:opa.address_std') END 
    END AS Notes,
    CASE WHEN lap.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM davelake.philly_opa_properties_public_pde opa
JOIN davelake.MasterEntity llc
    ON opa.owner_2 = llc.NameText
JOIN davelake.masteraddress addr
    ON opa.address_std = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress lap
    ON llc.MasterEntityID = lap.MasterEntityID
    AND addr.MasterAddressID = lap.MasterAddressID
    AND opa.parcel_number = lap.parcel_number
;
--updates
MERGE INTO davelake.MasterEntityAddress targ
    USING Case5
        ON targ.MasterEntityID = Case5.MasterEntityID
        AND targ.MasterAddressID = Case5.MasterAddressID
        AND targ.parcel_number = Case5.parcel_number
WHEN MATCHED AND Case5.NewEntry = 0 THEN 
    UPDATE SET targ.Notes = Case5.Notes;
--inserts
INSERT INTO davelake.MasterEntityAddress
SELECT 
    uuid() AS MasterEntityAddressID,
    Case5.MasterEntityID,
    Case5.MasterAddressID,
    Case5.parcel_number ,
    Case5.Notes
FROM  Case5
WHERE Case5.NewEntry = 1

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 177, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [61]:
--6.owner_2 associated wtih mailing_street
CREATE OR REPLACE TEMPORARY VIEW Case6
AS 
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    opa.parcel_number,
    lap.MasterEntityAddressID,
    CASE WHEN lap.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'opa.owner_1:opa.mailing_street'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN lap.Notes like '%opa.owner_1:opa.mailing_street%' THEN lap.Notes
            --new entry
            ELSE CONCAT(lap.Notes ,';opa.owner_1:opa.mailing_street') END 
    END AS Notes,
    CASE WHEN lap.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM davelake.philly_opa_properties_public_pde opa
JOIN davelake.MasterEntity llc
    ON opa.owner_2 = llc.NameText
JOIN davelake.masteraddress addr
    ON opa.mailing_street = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress lap
    ON llc.MasterEntityID = lap.MasterEntityID
    AND addr.MasterAddressID = lap.MasterAddressID
    AND opa.parcel_number = lap.parcel_number
;
--updates
MERGE INTO davelake.MasterEntityAddress targ
    USING Case6
        ON targ.MasterEntityID = Case6.MasterEntityID
        AND targ.MasterAddressID = Case6.MasterAddressID
        AND targ.parcel_number = Case6.parcel_number
WHEN MATCHED AND Case6.NewEntry = 0 THEN 
    UPDATE SET targ.Notes = Case6.Notes;
--inserts
INSERT INTO davelake.MasterEntityAddress
SELECT 
    uuid() AS MasterEntityAddressID,
    Case6.MasterEntityID,
    Case6.MasterAddressID,
    Case6.parcel_number ,
    Case6.Notes
FROM  Case6
WHERE Case6.NewEntry = 1

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 180, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Populate tables from philly_appeals

In [62]:
--get the distinct "LLC owners" from philly_appeals (opa_owner, primaryapellant)
INSERT INTO davelake.MasterEntity 
SELECT 
    uuid() AS MasterEntityID,
    DistinctOwners.NameText AS NameText
FROM 
    (
        --opa_owner
        SELECT 
            DISTINCT opa.opa_owner AS NameText
        FROM davelake.philly_appeals opa
        WHERE opa.opa_owner IS NOT NULL
        UNION
        --primaryappellant
        SELECT 
            DISTINCT opa.primaryappellant AS NameText
        FROM davelake.philly_appeals opa
        WHERE opa.primaryappellant IS NOT NULL
    ) DistinctOwners 
LEFT JOIN davelake.MasterEntity master
    ON DistinctOwners.NameText = master.NameText
--LLC doesn't yet exist in master table
WHERE master.NameText IS NULL
;

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 181, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [63]:
--now do distinct addresses from philly_appeals
--address col
INSERT INTO davelake.MasterAddress 
SELECT 
    uuid() AS MasterAddressID,
    DistinctAddresses.AddressText 
FROM 
    (
        --address col
        SELECT 
            DISTINCT pa.address AS AddressText
        FROM davelake.philly_appeals pa
        WHERE pa.address IS NOT NULL
    ) DistinctAddresses
--where the row does not yet exist
LEFT JOIN davelake.MasterAddress master
    ON DistinctAddresses.AddressText = master.AddressText
WHERE master.AddressText IS NULL

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 182, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [64]:
--now do MasterEntityAddress (junction table) for philly_appeals
--Case 1:  opa_owner associated with address
CREATE OR REPLACE TEMPORARY VIEW Case1
AS 
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    pa.opa_account_num,
    lap.MasterEntityAddressID,
    CASE WHEN lap.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'appeal.opa_owner:appeal.address'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN lap.Notes like '%appeal.opa_owner:appeal.address%' THEN lap.Notes
            --new entry
            ELSE CONCAT(lap.Notes ,';appeal.opa_owner:appeal.address') END 
    END AS Notes,
    CASE WHEN lap.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM davelake.philly_appeals pa
JOIN davelake.MasterEntity llc
    ON pa.opa_owner = llc.NameText
JOIN davelake.masteraddress addr
    ON pa.address = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress lap
    ON llc.MasterEntityID = lap.MasterEntityID
    AND addr.MasterAddressID = lap.MasterAddressID
    AND pa.opa_account_num = lap.parcel_number
;
--updates
MERGE INTO davelake.MasterEntityAddress targ
    USING Case1
        ON targ.MasterEntityID = Case1.MasterEntityID
        AND targ.MasterAddressID = Case1.MasterAddressID
        AND targ.parcel_number = Case1.opa_account_num
WHEN MATCHED AND Case1.NewEntry = 0 THEN 
    UPDATE SET targ.Notes = Case1.Notes;
--inserts
INSERT INTO davelake.MasterEntityAddress
SELECT 
    uuid() AS MasterEntityAddressID,
    Case1.MasterEntityID,
    Case1.MasterAddressID,
    Case1.opa_account_num ,
    Case1.Notes
FROM  Case1
WHERE Case1.NewEntry = 1

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 185, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [65]:
--Case 2:  primaryappellant associated with address
CREATE OR REPLACE TEMPORARY VIEW Case2
AS 
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    pa.opa_account_num,
    lap.MasterEntityAddressID,
    CASE WHEN lap.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'appeal.primaryappellant:appeal.address'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN lap.Notes like '%appeal.primaryappellant:appeal.address%' THEN lap.Notes
            --new entry
            ELSE CONCAT(lap.Notes ,';appeal.primaryappellant:appeal.address') END 
    END AS Notes,
    CASE WHEN lap.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM davelake.philly_appeals pa
JOIN davelake.MasterEntity llc
    ON pa.primaryappellant = llc.NameText
JOIN davelake.masteraddress addr
    ON pa.address = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress lap
    ON llc.MasterEntityID = lap.MasterEntityID
    AND addr.MasterAddressID = lap.MasterAddressID
    AND pa.opa_account_num = lap.parcel_number
;
--updates
MERGE INTO davelake.MasterEntityAddress targ
    USING Case2
        ON targ.MasterEntityID = Case2.MasterEntityID
        AND targ.MasterAddressID = Case2.MasterAddressID
        AND targ.parcel_number = Case2.opa_account_num
WHEN MATCHED AND Case2.NewEntry = 0 THEN 
    UPDATE SET targ.Notes = Case2.Notes;
--inserts
INSERT INTO davelake.MasterEntityAddress
SELECT 
    uuid() AS MasterEntityAddressID,
    Case2.MasterEntityID,
    Case2.MasterAddressID,
    Case2.opa_account_num ,
    Case2.Notes
FROM  Case2
WHERE Case2.NewEntry = 1

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 188, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Populate tables from philly_business_licenses

In [67]:
--get the distinct "LLC owners/entities" from business_licenses
--it would be nice if we could also include the opa_account_num to avoid a later view but 
--this means the distincts are not working (can't distinct on a distinct...dunno why)

CREATE OR REPLACE TEMPORARY VIEW BizLicEntities
AS 
SELECT  
    --opa_account_num,
    upper(business_name) as EntityName
    --'business_name' as EntityType
from davelake.philly_business_licenses 
where business_name is not null
UNION
SELECT 
    --opa_account_num,
    upper(opa_owner) as EntityName
    --'opa_owner' as EntityType
from davelake.philly_business_licenses 
where opa_owner is not null
UNION 
SELECT 
    --opa_account_num,
    upper(ownercontact1name) as EntityName
    --'ownercontact1name' as EntityType
from davelake.philly_business_licenses 
where ownercontact1name is not null
UNION 
SELECT 
    --opa_account_num,
    upper(ownercontact2name) as EntityName
    --'ownercontact2name' as EntityType
from davelake.philly_business_licenses 
where ownercontact2name is not null
UNION 
SELECT 
    --opa_account_num,
    upper(legalname) as EntityName
    --'legalname' as EntityType
from davelake.philly_business_licenses 
where legalname is not null
;


INSERT INTO davelake.MasterEntity 
SELECT 
    uuid() AS MasterEntityID,
    BizLicEntities.EntityName AS NameText
FROM BizLicEntities  
LEFT JOIN davelake.MasterEntity master
    ON BizLicEntities.EntityName = master.NameText
--LLC doesn't yet exist in master table
WHERE master.NameText IS NULL
;

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 196, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [69]:
--now do distinct addresses from business_licenses
CREATE OR REPLACE TEMPORARY VIEW BizLicAddresses
AS 
SELECT 
    --opa_account_num,
    address as NameText
    --'address' as EntityType
from davelake.philly_business_licenses 
where address is not null
UNION
SELECT 
    --opa_account_num,
    upper(business_mailing_address) as NameText
    --'business_mailing_address' as EntityType
from davelake.philly_business_licenses 
where business_mailing_address is not null
UNION 
SELECT 
    --opa_account_num,
    upper(ownercontact1mailingaddress) as NameText
    --'ownercontact1mailingaddress' as EntityType
from davelake.philly_business_licenses 
where ownercontact1mailingaddress is not null
UNION 
SELECT 
    --opa_account_num,
    upper(ownercontact2mailingaddress) as NameText
    --'ownercontact2mailingaddress' as EntityType
from davelake.philly_business_licenses 
where ownercontact2mailingaddress is not null
;

INSERT INTO davelake.MasterAddress 
SELECT 
    uuid() AS MasterAddressID,
    BizLicAddresses.NameText 
FROM BizLicAddresses 
--where the row does not yet exist
LEFT JOIN davelake.MasterAddress master
    ON BizLicAddresses.NameText = master.AddressText
WHERE master.AddressText IS NULL

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 199, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [106]:
--now do MasterEntityAddress (junction table) for business_licenses

--need a view similar to the above to get the distinct entities and parcel_numbers
CREATE OR REPLACE TEMPORARY VIEW BizLicOPAEntities
AS 
SELECT  
    opa_account_num,
    upper(trim(business_name)) as EntityName
    --'business_name' as EntityType
from davelake.philly_business_licenses 
where business_name is not null
UNION
SELECT 
    opa_account_num,
    upper(trim(opa_owner)) as EntityName
    --'opa_owner' as EntityType
from davelake.philly_business_licenses 
where opa_owner is not null
UNION 
SELECT 
    opa_account_num,
    upper(trim(ownercontact1name)) as EntityName
    --'ownercontact1name' as EntityType
from davelake.philly_business_licenses 
where ownercontact1name is not null
UNION 
SELECT 
    opa_account_num,
    upper(trim(ownercontact2name)) as EntityName
    --'ownercontact2name' as EntityType
from davelake.philly_business_licenses 
where ownercontact2name is not null
UNION 
SELECT 
    opa_account_num,
    upper(trim(legalname)) as EntityName
    --'legalname' as EntityType
from davelake.philly_business_licenses 
where legalname is not null
;

--need a view similar to the above to get the distinct addresses and parcel_numbers
CREATE OR REPLACE TEMPORARY VIEW BizLicOPAAddresses
AS 
SELECT 
    opa_account_num,
    address as NameText
    --'address' as EntityType
from davelake.philly_business_licenses 
where address is not null
UNION
SELECT 
    opa_account_num,
    upper(trim(business_mailing_address)) as NameText
    --'business_mailing_address' as EntityType
from davelake.philly_business_licenses 
where business_mailing_address is not null
UNION 
SELECT 
    opa_account_num,
    upper(trim(ownercontact1mailingaddress)) as NameText
    --'ownercontact1mailingaddress' as EntityType
from davelake.philly_business_licenses 
where ownercontact1mailingaddress is not null
UNION 
SELECT 
    opa_account_num,
    upper(trim(ownercontact2mailingaddress)) as NameText
    --'ownercontact2mailingaddress' as EntityType
from davelake.philly_business_licenses 
where ownercontact2mailingaddress is not null
;
--JOIN the 2 preceding views to get a flattened view of distinct OPAs, Entities, and Addresses
--these are all separate views to aid debugging
CREATE OR REPLACE TEMPORARY VIEW ExistingBizLicData
AS 
select e.opa_account_num, e.EntityName, a.NameText
from BizLicOPAEntities e 
JOIN BizLicOPAAddresses a
ON e.opa_account_num = a.opa_account_num
;

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 355, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [107]:
select * from BizLicOPAEntities WHERE EntityName IN ('HIM YIENG','YIENG HIM') order by opa_account_num;
select * from BizLicOPAAddresses WHERE opa_account_num IN (482269300,482269400,364427405) order by opa_account_num;
select * from ExistingBizLicData WHERE opa_account_num IN (482269300,482269400,364427405)

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 358, Finished, Available, Finished)

<Spark SQL result set with 6 rows and 2 fields>

<Spark SQL result set with 6 rows and 2 fields>

<Spark SQL result set with 12 rows and 3 fields>

In [109]:
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    Existing.opa_account_num,
    mea.MasterEntityAddressID,
    CASE WHEN mea.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'business_licenses.various:business_licenses.various'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN mea.Notes like '%business_licenses.various:business_licenses.various%' THEN mea.Notes
            --new entry
            ELSE CONCAT(mea.Notes ,';business_licenses.various:business_licenses.various') END 
    END AS Notes,
    CASE WHEN mea.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM ExistingBizLicData Existing
JOIN davelake.MasterEntity llc
    ON Existing.EntityName = llc.NameText
JOIN davelake.masteraddress addr
    ON Existing.NameText = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress mea
    ON llc.MasterEntityID = mea.MasterEntityID
    AND addr.MasterAddressID = mea.MasterAddressID
    AND Existing.opa_account_num = mea.parcel_number
WHERE Existing.opa_account_num IN (482269300,482269400,364427405);

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 360, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 6 fields>

In [ ]:
--now do MasterEntityAddress (junction table) for business_licenses

CREATE OR REPLACE TEMPORARY VIEW Case1
AS 
SELECT DISTINCT 
    llc.MasterEntityID,
    addr.MasterAddressID,
    pa.opa_account_num,
    lap.MasterEntityAddressID,
    CASE WHEN lap.MasterEntityAddressID 
        --new entry
        IS NULL THEN 'appeal.opa_owner:appeal.address'   
        --add to existing entry (ensuring idempotency)
        ELSE 
            --ignore it/entry exists already
            CASE WHEN lap.Notes like '%appeal.opa_owner:appeal.address%' THEN lap.Notes
            --new entry
            ELSE CONCAT(lap.Notes ,';appeal.opa_owner:appeal.address') END 
    END AS Notes,
    CASE WHEN lap.MasterEntityAddressID IS NULL THEN 1 ELSE 0 END AS NewEntry
FROM DistinctBizLicEntities Entities
JOIN davelake.MasterEntity llc
    ON Entities.EntityName = llc.NameText
JOIN davelake.masteraddress addr
    ON pa.address = addr.AddressText
LEFT JOIN davelake.MasterEntityAddress lap
    ON llc.MasterEntityID = lap.MasterEntityID
    AND addr.MasterAddressID = lap.MasterAddressID
    AND pa.opa_account_num = lap.parcel_number
;
--updates
MERGE INTO davelake.MasterEntityAddress targ
    USING Case1
        ON targ.MasterEntityID = Case1.MasterEntityID
        AND targ.MasterAddressID = Case1.MasterAddressID
        AND targ.parcel_number = Case1.opa_account_num
WHEN MATCHED AND Case1.NewEntry = 0 THEN 
    UPDATE SET targ.Notes = Case1.Notes;
--inserts
INSERT INTO davelake.MasterEntityAddress
SELECT 
    uuid() AS MasterEntityAddressID,
    Case1.MasterEntityID,
    Case1.MasterAddressID,
    Case1.opa_account_num ,
    Case1.Notes
FROM  Case1
WHERE Case1.NewEntry = 1

## Tester Block

In [70]:
--sanity checker, looking for dups
select NameText, count(*) from davelake.MasterEntity GROUP BY NameText Having count(*) > 1;
select MasterEntityID, count(*) from davelake.MasterEntity GROUP BY MasterEntityID Having count(*) > 1;
select AddressText, count(*) from davelake.Masteraddress GROUP BY AddressText Having count(*) > 1;
select MasterAddressID, count(*) from davelake.Masteraddress GROUP BY MasterAddressID Having count(*) > 1;
--there may be dups if the parcel_number is null
select MasterEntityID, MasterAddressID, parcel_number, count(*) from davelake.MasterEntityAddress GROUP BY MasterEntityID, MasterAddressID, parcel_number Having count(*) > 1;
select MasterEntityAddressID, count(*) from davelake.MasterEntityAddress GROUP BY MasterEntityAddressID Having count(*) > 1;

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 205, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 2 fields>

<Spark SQL result set with 0 rows and 2 fields>

<Spark SQL result set with 0 rows and 2 fields>

<Spark SQL result set with 0 rows and 2 fields>

<Spark SQL result set with 4 rows and 4 fields>

<Spark SQL result set with 0 rows and 2 fields>

In [9]:
select * from MasterEntity where MasterEntityID in (select MasterEntityID from davelake.MasterEntityAddress GROUP BY MasterEntityID, MasterAddressID, parcel_number Having count(*) > 1)

StatementMeta(, 0dd54b87-1542-4175-ba98-b3dca94b8968, 28, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 2 fields>

In [28]:
--tester block
select * from davelake.MasterEntity where MasterEntityID = '9d64b9f1-8776-4922-89ed-c5b1994b5286' limit 10;
select * from davelake.masteraddress where addresstext = '2837 KENSINGTON AVE' limit 10;
select * from davelake.MasterEntityAddress where MasterAddressID = '552b014f-6bd9-4afa-91e3-22fed0ac566a' ;
select * from davelake.MasterEntityAddress where MasterEntityID = '9d64b9f1-8776-4922-89ed-c5b1994b5286' ;
select * from davelake.masteraddress where masteraddressid = '8acb8b1f-3a08-47dd-a375-67c8ce025606'


StatementMeta(, e5ea8fac-7a58-4ffd-8bf9-15d826456ffa, 103, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 2 fields>

<Spark SQL result set with 1 rows and 2 fields>

<Spark SQL result set with 0 rows and 5 fields>

<Spark SQL result set with 0 rows and 5 fields>

<Spark SQL result set with 0 rows and 2 fields>

## TODO block

In [4]:
%%sql 

/* Other tables with addresses/business names that could indicate the LLC/owner/property address:
(TODO):
 
*/


select 
    opa_account_num, address, upper(business_mailing_address) AS business_mailing_address, 
    upper(ownercontact1mailingaddress) AS ownercontact1mailingaddress, upper(ownercontact2mailingaddress) AS ownercontact2mailingaddress,
    upper(business_name) as business_name, upper(opa_owner) as opa_owner, upper(ownercontact1name) ownercontact1name,upper(ownercontact2name) ownercontact2name, upper(legalname) as legalname
from davelake.philly_business_licenses 
--where opa_owner = 'A KENSINGTON JOINT LLC' 
limit 200;

--there is no parcel_num for this table
select upper(companyname) as companyname, upper(legalfirstname) as legalfirstname, upper(legallastname) legallastname ,
upper(ownercontact1name) ownercontact1name,
upper(ownercontact1mailingaddress) ownercontact1mailingaddress, upper(ownercontact2mailingaddress) ownercontact2mailingaddress, upper(ownercontact2name) ownercontact2name
from davelake.philly_com_act_licenses LIMIT 10;


StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 22, Finished, Available, Finished)

<Spark SQL result set with 200 rows and 11 fields>

<Spark SQL result set with 100 rows and 24 fields>

<Spark SQL result set with 10 rows and 7 fields>

In [2]:
%%sql 

/* Other tables with addresses/business names that could indicate the LLC/owner/property address:
(TODO):
 
*/
select address, opa_account_num, opa_owner 
from davelake.philly_case_investigations limit 2;

--look at contractoraddress1 carefully
select * from davelake.philly_demolitions limit 10;
select address, applicantname, contractoraddress1, contractoraddress2, contractorname, opa_owner
--JPC GROUP INC is the city's contractor , but also does private 
from davelake.philly_demolitions limit 10;

--todo
select * from davelake.philly_permits limit 10;
select address, contractoraddress1, contractoraddress2,contractorname, opa_owner, opa_account_num  from davelake.philly_permits LIMIT 10;
--opa_owner should be the same in opa base table, but might not be if there is an error

--todo
select * from davelake.philly_violations limit 10;
select address, opa_owner, opa_account_num from davelake.philly_violations limit 10;

StatementMeta(, cffc0ca2-1d06-48da-943e-5b19461e7c71, 16, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 3 fields>

<Spark SQL result set with 10 rows and 36 fields>

<Spark SQL result set with 10 rows and 6 fields>

<Spark SQL result set with 10 rows and 34 fields>

<Spark SQL result set with 10 rows and 6 fields>

<Spark SQL result set with 10 rows and 34 fields>

<Spark SQL result set with 10 rows and 3 fields>

In [30]:
%%sql 

/* Other tables with addresses/business names that could indicate the LLC/owner/property address:
(TODO):
 
*/
select opa_number, street_address,mailing_address, owner, co_owner from davelake.philly_real_estate_tax_balances limit 10;
--likely need to split grantees/grantors on the semicolon
select grantees, grantors, opa_account_num, street_address from davelake.philly_rtt_summary limit 10;

StatementMeta(, 71f05189-1c4e-44ba-a0b2-147011c87d65, 80, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 5 fields>

<Spark SQL result set with 10 rows and 4 fields>

In [31]:
%%sql 

select 
    parent_owner.owner_1 as ParentOwner_1,
    parent_owner.parcel_number as ParentParcelNumber,
    other_properties.location,
    other_properties.parcel_number,
    other_properties.assessment_date,
    other_properties.owner_1,
    other_properties.owner_2,
    other_properties.homestead_exemption,
    other_properties.recording_date,
    other_properties.sale_date,
    other_properties.sale_price,
    other_properties.zoning
from davelake.philly_opa_properties_public_pde parent_owner
left join davelake.philly_opa_properties_public_pde other_properties
where parent_owner.location = '2837 KENSINGTON AVE'

StatementMeta(, 463061cd-d4d8-4b7b-8796-c73e588b166d, 37, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 12 fields>

## WIP

Deed transactions...one row per transaction

In [3]:
%%sql 

select 
    rtt.adjusted_assessed_value,
    rtt.adjusted_cash_consideration,
    rtt.adjusted_fair_market_value,
    rtt.adjusted_local_tax_amount,
    rtt.assessed_value,
    rtt.document_date,
    rtt.document_id,
    rtt.document_type,
    rtt.grantees,
    rtt.grantors,
    rtt.legal_remarks,
    rtt.recording_date
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_rtt_summary rtt 
    on opa.parcel_number = rtt.opa_account_num
where opa.location = '2837 KENSINGTON AVE'
order by rtt.document_date desc
;

StatementMeta(, 463061cd-d4d8-4b7b-8796-c73e588b166d, 4, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 12 fields>

Assessments.  One row per assessment

In [5]:
%%sql

select 
    assess.market_value,
    assess.taxable_building,
    assess.taxable_land,
    assess.year,
    assess.exempt_building,
    assess.exempt_land
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_assessments assess
    on opa.parcel_number = assess.parcel_number
where opa.location = '2837 KENSINGTON AVE'
order by assess.year desc
limit 10;

StatementMeta(, 463061cd-d4d8-4b7b-8796-c73e588b166d, 6, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 6 fields>

In [27]:
%%sql

select 
    app.appealgrounds,
    app.appealstatus,
    app.decision,
    app.decisiondate,
    app.opa_owner,
    app.primaryappellant
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_appeals app
    on opa.parcel_number = app.opa_account_num
where opa.location = '2837 KENSINGTON AVE'

StatementMeta(, 349b3e0d-791f-43c4-b325-b37d1aea8b51, 28, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 38 fields>

In [31]:
%%sql 

select 
    cases.casenumber,
    cases.casepriority,
    cases.caseresponsibility,
    cases.casetype,
    cases.investigationcompleted,
    cases.investigationstatus,
    cases.investigationtype
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_case_investigations cases
    on opa.parcel_number = cases.opa_account_num
where opa.location = '2837 KENSINGTON AVE'
order by cases.investigationcompleted desc;

StatementMeta(, 349b3e0d-791f-43c4-b325-b37d1aea8b51, 32, Finished, Available, Finished)

<Spark SQL result set with 20 rows and 33 fields>

In [ ]:
%%sql 

/* Other tables with addresses/business names that could indicate the LLC/owner/property address:
(TODO):
 
*/
select opa_number, street_address,mailing_address, owner, co_owner from davelake.philly_real_estate_tax_balances limit 10;
--likely need to split grantees/grantors on the semicolon
select grantees, grantors, opa_account_num, street_address from davelake.philly_rtt_summary limit 10;

In [ ]:
select 
    rtt.adjusted_assessed_value,
    rtt.adjusted_cash_consideration,
    rtt.adjusted_fair_market_value,
    rtt.adjusted_local_tax_amount,
    rtt.assessed_value,
    rtt.document_date,
    rtt.document_id,
    rtt.document_type,
    rtt.grantees,
    rtt.grantors,
    rtt.legal_remarks,
    rtt.recording_date
from davelake.philly_opa_properties_public_pde opa
left join davelake.philly_rtt_summary rtt 
    on opa.parcel_number = rtt.opa_account_num
where opa.location = '2837 KENSINGTON AVE'
order by rtt.document_date desc